<a href="https://colab.research.google.com/github/Chaitya0623/Infiheal_Codes/blob/main/Infiheal_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/Part2-1.csv')
df

,text
0,you re definitely wrong i ve had it explained...
1,it is your problem actually because you made t...
2,it s not my own story though it s literally...
3,thank you for for sharing a reminder if you ...
4,forgive me if i m stating the obvious or sound...
...,...
7850,it was also confusing because at other times h...
7851,i relate to this so much my ndad found a pare...
7852,when my oldest was a toddler her favorite game...
7853,ugh god that s the killer i remember feeling ...


In [ ]:
!pip install transformers
!pip install sentence_transformers
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=82b6d39fa2b44f61d4918cc619e62231923efe29b48aca5d0c660259672af67b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",device=0)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import gc

# Load the SBERT model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
candidate_labels = ['Narcissistic Trauma', 'Vicarious Trauma',
       'Traumatic Grief', 'Divorce/parents fighting',
       'Community Violence + Terrorism',
       'Trauma caused due to domestic violence', 'Sexual Abuse ',
       'Family Secrets and Trauma', 'Accidents', 'Abandonment/ Adoption',
       'War', 'Poverty', 'Sickness (person was sick a lot)',
       'Authoritarian / strict parenting', 'Religious cult', 'Neglect',
       'Bullying', 'Enmeshment / covert incest', 'Parental Alienation',
       'Betrayal Trauma', 'Inconsistent parenting', 'Conditional Love ']

lower_cutoff = 0.3

df['labels'] = ''
df['multilabel'] = ''

for i, text in enumerate(df['text']):
  gc.collect()
  output = classifier(text, candidate_labels,multi_label=True)
  predicted_labels = [label for label in output['labels'][:3]]
  text_embedding = model.encode(text)
  label_embeddings = model.encode(predicted_labels)
  # Compute similarity scores using the model's similarity function
  similarity_scores = cosine_similarity([text_embedding], label_embeddings)[0]
  flag = 0
  for sim in similarity_scores:
      if sim > lower_cutoff:
          flag = 1
          break
  if flag == 0:
    df['labels'][i] = 'Insufficient'
    df['multilabel'][i] = 'Insufficient'
  else:
    df['labels'][i] = predicted_labels[similarity_scores.argmax()]
    df['multilabel'][i] = predicted_labels
  if i%50==0:
    print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850


In [ ]:
df

,text,labels,multilabel
0,you re definitely wrong i ve had it explained...,Insufficient,Insufficient
1,it is your problem actually because you made t...,Insufficient,Insufficient
2,it s not my own story though it s literally...,Narcissistic Trauma,"[Traumatic Grief, Narcissistic Trauma, Family ..."
3,thank you for for sharing a reminder if you ...,Insufficient,Insufficient
4,forgive me if i m stating the obvious or sound...,Insufficient,Insufficient
...,...,...,...
7850,it was also confusing because at other times h...,Insufficient,Insufficient
7851,i relate to this so much my ndad found a pare...,Inconsistent parenting,"[Inconsistent parenting, Bullying, Vicarious T..."
7852,when my oldest was a toddler her favorite game...,Parental Alienation,"[Vicarious Trauma, Betrayal Trauma, Parental A..."
7853,ugh god that s the killer i remember feeling ...,Insufficient,Insufficient


In [ ]:
df.to_csv('Part2-1_final.csv')